In [3]:
#Importa os pacotes necessários

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
plt.style.use('dark_background')

#### Dicionário de Variáveis utilizadas
|Chave|Descrição|Função|
|-----|-----|-----|
|**Identificação**|
|Ano| Ano de referência|Controle|
|Trimestre| Trimestre de referência|Controle|
|UF| Estado|Controle|
|Capital| Capital da UF|Controle|
|UPA| Unidade Primária de Amostragem (UF(2) + número squencial(6) + DV(1))|Controle|
|V1008| Número do Domicílio|Controle|
|V1014| Painel - Grupo de Controle|Controle|
|V1016| Número de entrevista (1 a 5)|Controle|
|V2003| Número de ordem|Controle|
|**Características**|
|V2007| Sexo|Característica|
|V2008| Dia Nascimento|Característica|
|V20081| Mês nascimento|Característica|
|V20082| Ano nascimento|Característica|
|V2010| Raça|Característica|
|**Variáveis exploradas**|
| VD3004 | Nível de instrução mais elevado alcançado | Medida |
| VD4001 | Condição em relação à força de trabalho | Medida |
| VD4002 | Condição de ocupação | Medida |
| VD4004 | Subocupação | Medida |
| VD4005 | Pessoas desalentadas | Medida |
|**Auxiliares**|
|V1028| Peso do domicílio e das pessoas sem calibração|Expansão|
|V1029| Peso do domicílio e das pessoas com calibração|Expansão|
|V1033| Projeção da população por sexo e idade|Expansão|
|posest|Domínios de projeção geográficos|Expansão|

 Chave para pessoa: UPA + V1008 + V1014  + V2003 + V2007 + V2008 + V20081 + V20082

 **PARA ESTIMAÇÃO DA POPULAÇÃO, DEVE-SE USAR AS VARIÁVEIS: V1033 e posest.**



#### Importação e tratamento dos microdados

Importação dos microdados

In [4]:
#determina qual diretório usar para cada sistema operacional.
import os
if os.name == 'nt': #windows
    diretorio = 'C:\\Users\\est.matheus.brasil\\Documents\\Microdados PNAD Contínua\\'
    enconding = 'latin_1'
else: #linux(ou mac)
    diretorio = '/home/matheus/Microdados PNAD/'
    enconding = 'utf_8_sig'

#Leitura de todas as bases com as variáveis selecionadas e criação de variáveis dinamicamente.

colunas_selecionadas = ['Ano', 'Trimestre', 'UPA', 'UF', 'V1008', 'V1014', 'V2007', 'V2008', 'V20081', 'V20082', 'V2010', #Controle e identificação
                        'VD3004','VD4001','VD4002','VD4004','VD4005', #Variáveis de interesse
                        'V1028', 'V1029', 'V1033', 'posest', 'posest_sxi'] #auxiliares

for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding=enconding)
        if 2019 < ano < 2022:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding=enconding)
        if ano == 2022 and tri < 3:
            print(ano, tri)
            globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding=enconding)

2019 4


<ipython-input-4-e4cb19f821b0>:20: DtypeWarning: Columns (197) have mixed types. Specify dtype option on import or set low_memory=False.
  globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{diretorio}{ano}_{tri}.csv', usecols=colunas_selecionadas, encoding=enconding)


2020 1
2020 2
2020 3
2020 4
2021 1
2021 2
2021 3
2021 4
2022 1
2022 2


In [5]:
#Função para automalizar importação das bases da pnad com as variáveis desejadas

def importa_pnad(path, ano_inicial, ano_final, colunas):
    import traceback
    '''
    path: somente o diretório onde os arquivos estão armazenados.
    ano_inicial: ano inicial do período analisado.
    ano_final: ano final do período analisado.
    '''
    try:    
        for ano in range(ano_inicial, ano_final+1):
            for tri in range(1, 5):
                if ano == 2019  and tri > 3:
                    print(ano, tri)
                    globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{path}{ano}_{tri}.csv', usecols=colunas, encoding='latin-1')
                if 2019 < ano < 2022:
                    print(ano, tri)
                    globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{path}{ano}_{tri}.csv', usecols=colunas, encoding='latin-1')
                if ano == 2022 and tri < 3:
                    print(ano, tri)
                    globals()[f'pnad_{ano}_{tri}'] = pd.read_csv(fr'{path}{ano}_{tri}.csv', usecols=colunas, encoding='latin-1')
        print('Dados importados com sucesso!')
    except Exception:
        traceback.print_exc() #mostra erro em caso de problema na importação

Tratamento

In [6]:
#Conversão das variável V2007(sexo) para inteira para criar ID do indivíduo. (Homem = 0, Mulher = 1)
for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)
        if 2019 < ano < 2022:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)
        if ano == 2022 and tri < 3:
            globals()[f'pnad_{ano}_{tri}']['V2007'] = np.where(globals()[f'pnad_{ano}_{tri}']['V2007'] == 'Homem', 0, 1)


In [8]:
#Organização das variáveis e criação da chave de ID do indivíduo em cada dataframe.

for ano in range(2019, 2023):
    for tri in range(1, 5):
        if ano == 2019  and tri > 3:
            globals()[f'pnad_{ano}_{tri}_educ_sit'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'VD3004': globals()[f'pnad_{ano}_{tri}']['VD3004'],
                                                        'VD4001': globals()[f'pnad_{ano}_{tri}']['VD4001'],
                                                        'VD4002': globals()[f'pnad_{ano}_{tri}']['VD4002'],
                                                        'VD4004': globals()[f'pnad_{ano}_{tri}']['VD4004'],
                                                        'VD4005': globals()[f'pnad_{ano}_{tri}']['VD4005'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })
        if 2019 < ano < 2022:
            globals()[f'pnad_{ano}_{tri}_educ_sit'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'VD3004': globals()[f'pnad_{ano}_{tri}']['VD3004'],
                                                        'VD4001': globals()[f'pnad_{ano}_{tri}']['VD4001'],
                                                        'VD4002': globals()[f'pnad_{ano}_{tri}']['VD4002'],
                                                        'VD4004': globals()[f'pnad_{ano}_{tri}']['VD4004'],
                                                        'VD4005': globals()[f'pnad_{ano}_{tri}']['VD4005'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })
        if ano == 2022 and tri < 3:
            globals()[f'pnad_{ano}_{tri}_educ_sit'] = pd.DataFrame(
                                                        {'Ano':globals()[f'pnad_{ano}_{tri}']['Ano'], 
                                                        'Trimestre':globals()[f'pnad_{ano}_{tri}']['Trimestre'],
                                                        'UPA':globals()[f'pnad_{ano}_{tri}']['UPA'],
                                                        'UF':globals()[f'pnad_{ano}_{tri}']['UF'],
                                                        'V2007':globals()[f'pnad_{ano}_{tri}']['V2007'],  
                                                        'ID': globals()[f'pnad_{ano}_{tri}']['UPA'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1008'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V1014'] + 
                                                              globals()[f'pnad_{ano}_{tri}']['V2007'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V2008'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20081'] +
                                                              globals()[f'pnad_{ano}_{tri}']['V20082'],
                                                        'V2010': globals()[f'pnad_{ano}_{tri}']['V2010'],
                                                        'VD3004': globals()[f'pnad_{ano}_{tri}']['VD3004'],
                                                        'VD4001': globals()[f'pnad_{ano}_{tri}']['VD4001'],
                                                        'VD4002': globals()[f'pnad_{ano}_{tri}']['VD4002'],
                                                        'VD4004': globals()[f'pnad_{ano}_{tri}']['VD4004'],
                                                        'VD4005': globals()[f'pnad_{ano}_{tri}']['VD4005'],
                                                        'V1028': globals()[f'pnad_{ano}_{tri}']['V1028'],
                                                        'V1029': globals()[f'pnad_{ano}_{tri}']['V1029'],
                                                        'V1033': globals()[f'pnad_{ano}_{tri}']['V1033'],
                                                        'posest': globals()[f'pnad_{ano}_{tri}']['posest'],
                                                        'posest_sxi': globals()[f'pnad_{ano}_{tri}']['posest_sxi']
                                                      })

In [9]:
#Concatena todos os dataframes de cada ano e trimestre em uma unica tabela.

pnadc_completa = pd.concat([pnad_2019_4_educ_sit, 
                            pnad_2020_1_educ_sit, 
                            pnad_2020_2_educ_sit, 
                            pnad_2020_3_educ_sit, 
                            pnad_2020_4_educ_sit, 
                            pnad_2021_1_educ_sit, 
                            pnad_2021_2_educ_sit, 
                            pnad_2021_3_educ_sit, 
                            pnad_2021_4_educ_sit, 
                            pnad_2022_1_educ_sit, 
                            pnad_2022_2_educ_sit])

In [10]:
pnadc_completa

,Ano,Trimestre,UPA,UF,V2007,ID,V2010,VD3004,VD4001,VD4002,VD4004,VD4005,V1028,V1029,V1033,posest,posest_sxi
0,2019,4,110000016,Rondônia,1,110002003,Parda,Médio completo ou equivalente,Pessoas na força de trabalho,Pessoas ocupadas,NaN,NaN,145.628790,532471,6982307,111,210
1,2019,4,110000016,Rondônia,1,110002034,Parda,Médio completo ou equivalente,Pessoas fora da força de trabalho,NaN,NaN,NaN,145.628790,532471,8485527,111,206
2,2019,4,110000016,Rondônia,1,110002051,Parda,Médio completo ou equivalente,Pessoas fora da força de trabalho,NaN,NaN,NaN,145.628790,532471,9316281,111,204
3,2019,4,110000016,Rondônia,0,110002077,Parda,Sem instrução e menos de 1 ano de estudo,NaN,NaN,NaN,NaN,145.628790,532471,7432741,111,102
4,2019,4,110000016,Rondônia,0,110002012,Parda,Fundamental incompleto ou equivalente,Pessoas fora da força de trabalho,NaN,NaN,NaN,134.017538,532471,5214819,111,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482113,2022,2,530057544,Distrito Federal,0,530059585,Parda,Médio completo ou equivalente,Pessoas na força de trabalho,Pessoas ocupadas,NaN,NaN,81.626590,3125014,8442398,531,107
482114,2022,2,530057544,Distrito Federal,0,530059543,Preta,Fundamental incompleto ou equivalente,Pessoas na força de trabalho,Pessoas ocupadas,NaN,NaN,72.323502,3125014,3560554,531,114
482115,2022,2,530057544,Distrito Federal,0,530059578,Preta,Superior completo,Pessoas na força de trabalho,Pessoas ocupadas,NaN,NaN,72.323502,3125014,8409440,531,108
482116,2022,2,530057544,Distrito Federal,1,530059586,Branca,Médio completo ou equivalente,Pessoas fora da força de trabalho,NaN,NaN,NaN,72.323502,3125014,8558427,531,207
